# Lab objective : Implementing AutoGPT using LangChain

### Installing Langchain and other required Libraries for this lab

In [1]:
!pip install langchain==0.1.11 langchain_community langchain_openai faiss-cpu langchain-experimental google-search-results


INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.5 MB 544.7 kB/s eta 0:00:27
   ---------------------------------------- 0.1/14.5 MB 944.1 kB/s eta 0:00:16
   ---------------------------------------- 0.2/14.5 MB 984.6 kB/s eta 0:00:15
    --------------------------------------- 0.3/14.5 MB 1.4 MB/s eta 0:00:11
   - -------------------------------------- 0.4/14.5 MB 1.6 MB/s eta 0:00:09
   - -------------------------------------- 0.7/14.5 MB 2.1 MB/s eta 0:00:07
   -- ------------------------------------- 0.9/14.5 MB 2.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.0/14.5 MB 2.6 MB/s eta 0:00:06
   --- ------------------------------------ 1.2/14.5 MB 2.8 MB/s eta 0:00:05
  

### Setting Up Tools for File Management and Search Functionality with SerpAPI

**Search Tool Setup**: Initializes a SerpAPIWrapper for web searches, configured with a specific API key, and creates a search tool with a function to execute searches.

**File Management Tools**: Sets up tools for reading and writing files using the WriteFileTool and ReadFileTool classes.

**Tool Collection**: Combines the search, write, and read tools into a list, ready for integration into larger workflows or applications.





In [2]:
from langchain.agents import Tool
from langchain_community.tools.file_management.read import ReadFileTool
from langchain_community.tools.file_management.write import WriteFileTool
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper(serpapi_api_key="serp_api")
tools = [
 Tool(
  name="search",
 func=search.run,
 description="useful for when you need to answer questions about current events. You should ask targeted questions",
 ),
WriteFileTool(),
ReadFileTool(),
]

### Initializing an In-Memory Document Store with FAISS and OpenAI Embeddings
1. **Environment and Embedding Setup**: Configures the OpenAI API key and initializes an OpenAI embeddings model with a FAISS index tailored for vector storage and similarity calculations.

2. **Vector Store Initialization**: Establishes a FAISS-based vector store, combining the embeddings model and index with an in-memory document store for efficient data management.

In [3]:
from langchain.docstore import InMemoryDocstore
import faiss
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

import os
os.environ['OPENAI_API_KEY']="your_api_key"

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})


### Configuring an AutoGPT Agent with Embedded Memory and Tools

1. **Agent Initialization**: Constructs an `AutoGPT` agent named "Tom" with an assistant role, integrating a list of tools and a chat model (`ChatOpenAI`), using a vector store as memory for retrieving context.

2. **Verbose Setting**: Enables verbose output for the agent’s processing chain to provide detailed feedback and logging during operations.

In [4]:
from langchain_experimental.autonomous_agents import AutoGPT
from langchain_openai import ChatOpenAI

agent = AutoGPT.from_llm_and_tools(
  ai_name="Tom",
  ai_role="Assistant",
 tools=tools,
 llm=ChatOpenAI(),
 memory=vectorstore.as_retriever(),
)
# Set verbose to be true
agent.chain.verbose = True



Once we execute the below code, AutoGPT wrote the weather report of in “weather_report_new_york.txt” file because we have given the command of “write a weather report for New york today”.


In [5]:
agent.run(["write a weather report for New york today"])




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. write a weather report for New york today


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: useful for when you need to answer questions about current events. You should ask targeted questions, args json schema: {"tool_input": {"type": "string"}}
2. write_file: Write file to disk, args json sch


> Finished chain.
{
    "thoughts": {
        "text": "I have gathered the current weather information for New York. Now, I should proceed to write the weather report based on the details I found.",
        "reasoning": "Writing the weather report will provide a summary of the current conditions in New York.",
        "plan": [
            "Compose a weather report for New York based on the gathered information."
        ],
        "criticism": "None",
        "speak": "I will now write a weather report for New York based on the current conditions."
    },
    "command": {
        "name": "write_file",
        "args": {
            "file_path": "weather_report_new_york.txt",
            "text": "Weather Report for New York:\n\n- Location: New York, NY\n- Date: Saturday 11:00 AM\n- Temperature: 59°F\n- Weather: Mostly sunny\n- Precipitation: 0%\n- Humidity: 44%\n- Wind: 7 mph",
            "append": false
        }
    }
}


> Entering new LLMChain chain...
Prompt after formatting:
Sys

'Weather report for New York has been successfully written and all objectives have been completed.'